In [2]:
%pip install ultralytics opencv-python

In [3]:
%pip install --upgrade pip


Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import cv2  # For video processing
import shutil  # For moving files


In [4]:
# Define paths
video_path = r'C:\Users\mdfra\Downloads\deepfake-detection'  # Path to test videos
frames_path = 'frames/test/'  # Output frames directory

# Create the output directory if it doesn't exist
os.makedirs(frames_path, exist_ok=True)

# Function to extract frames from video
def extract_frames(video_file, output_dir):
    cap = cv2.VideoCapture(video_file)
    count = 0
    while cap.isOpened():
        ret, frame = cap.read()
        if not ret:
            break
        # Save every 10th frame
        if count % 10 == 0:
            cv2.imwrite(os.path.join(output_dir, f'frame_{count}.jpg'), frame)
        count += 1
    cap.release()

# Extract frames from all videos in the directory
for video_file in os.listdir(video_path):
    if video_file.endswith('.mp4'):  # Adjust the extension based on your video format
        extract_frames(os.path.join(video_path, video_file), frames_path)


In [9]:
import os

# Path where frames are saved (make sure this path is defined correctly)
frames_path = r"C:\Users\mdfra\Downloads\deepfake-detection\deepfake-training\frames"  # Adjust this to your actual path

# List of missing labels (you should check for this)
missing_labels = [
    'frame_262.txt', 'frame_474.txt', 'frame_444.txt', 'frame_36.txt',
    'frame_632.txt', 'frame_358.txt', 'frame_33.txt', 'frame_101.txt',
    'frame_392.txt', 'frame_92.txt', 'frame_235.txt', 'frame_304.txt'
]

# Assume all missing frames are classified as real (class_id = 0)
class_id = 0  # Change this based on your classification

# Ensure the directory exists
os.makedirs(frames_path, exist_ok=True)

# Create label files for missing frames
for missing_label in missing_labels:
    # Get the base filename without extension
    base_name = os.path.splitext(missing_label)[0]
    
    # Create a new label file in the same directory
    label_file_path = os.path.join(frames_path, missing_label)
    
    # Write the default label for the real class
    try:
        with open(label_file_path, 'w') as f:
            f.write(f"{class_id} 0.5 0.5 1 1\n")  # Adjust coordinates if necessary
    except FileNotFoundError as e:
        print(f"Error creating file {label_file_path}: {e}")

print(f"Created label files for: {missing_labels}")

# Optional: Check for missing labels again
missing_labels_updated = []

# List all files in the frames directory
for frame_file in os.listdir(frames_path):
    if frame_file.endswith('.jpg'):  # or .png based on your image type
        label_filename = os.path.splitext(frame_file)[0] + '.txt'
        if label_filename not in os.listdir(frames_path):
            missing_labels_updated.append(label_filename)

if missing_labels_updated:
    print(f"Still missing label files for: {missing_labels_updated}")
else:
    print("All label files are now present.")


Created label files for: ['frame_262.txt', 'frame_474.txt', 'frame_444.txt', 'frame_36.txt', 'frame_632.txt', 'frame_358.txt', 'frame_33.txt', 'frame_101.txt', 'frame_392.txt', 'frame_92.txt', 'frame_235.txt', 'frame_304.txt']
All label files are now present.


In [10]:
data_yaml_path = 'data.yaml'

data_yaml_content = """
train: frames/YouTube-real/
val: frames/Celeb-real/
test: frames/test/

nc: 2
names: ['real', 'fake']
"""

with open(data_yaml_path, 'w') as f:
    f.write(data_yaml_content)

print(f"Data configuration file created at: {data_yaml_path}")


Data configuration file created at: data.yaml


In [11]:
%pip install ultralytics


Note: you may need to restart the kernel to use updated packages.


In [12]:
from ultralytics import YOLO

# Load YOLOv8n model (replace 'yolov8n.pt' with another model if desired)
model = YOLO('yolov8n.pt')


In [13]:
train: r'C:/Users/mdfra/Downloads/deepfake-detection/deepfake-training/frames/Celeb-real'
val: r'C:/Users/mdfra/Downloads/deepfake-detection/deepfake-training/frames/test'

nc: 2
names: ['real', 'fake']  # Update the class names according to your dataset


In [15]:
# Define the dataset configuration
dataset_config = {
    'path': r'C:/Users/mdfra/Downloads/deepfake-detection/deep-fake-dataset',  # Root directory of your dataset
    'train': r'Celeb-real',  # Folder with training images
    'val': r'test',          # Folder with validation images
    'names': {
        0: 'real',
        1: 'fake'
    }
}

# Accessing elements in the configuration
print(dataset_config['path'])
print(dataset_config['train'])
print(dataset_config['val'])
print(dataset_config['names'][0])  # Output: real
print(dataset_config['names'][1])  # Output: fake

C:/Users/mdfra/Downloads/deepfake-detection/deep-fake-dataset
Celeb-real
test
real
fake


In [1]:
%pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu95


Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu95
  Using cached torchaudio-2.5.1-cp311-cp311-win_amd64.whl.metadata (6.5 kB)
INFO: pip is looking at multiple versions of torchaudio to determine which version is compatible with other requirements. This could take a while.
   ---------------------------------------- 0.0/2.4 MB ? eta -:--:--
   ---------------------------------- ----- 2.1/2.4 MB 14.7 MB/s eta 0:00:01
   ---------------------------------------- 2.4/2.4 MB 14.0 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [1]:
# Import required libraries
from ultralytics import YOLO
import os
import cv2

# Define paths for dataset and frames
dataset_path = r"C:\Users\mdfra\Downloads\deepfake-detection\deep-fake-dataset"
output_path = r"C:\Users\mdfra\Downloads\deepfake-detection\frames"

# Function to extract frames from videos
def extract_frames(video_path, output_folder, frame_rate=1):
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
    
    cap = cv2.VideoCapture(video_path)
    frame_count = 0
    success, frame = cap.read()
    
    while success:
        if frame_count % frame_rate == 0:
            frame_filename = os.path.join(output_folder, f"frame_{frame_count}.jpg")
            cv2.imwrite(frame_filename, frame)
        success, frame = cap.read()
        frame_count += 1
    
    cap.release()

# Extract frames for each category
video_categories = ["test", "YouTube-real", "Celeb-real", "Celeb-Youtube-fake"]
for category in video_categories:
    category_path = os.path.join(dataset_path, category)
    category_output = os.path.join(output_path, category)
    for video_file in os.listdir(category_path):
        if video_file.endswith(('.mp4', '.avi')):
            video_path = os.path.join(category_path, video_file)
            extract_frames(video_path, category_output)

# Prepare YOLO model and data.yaml path
model = YOLO("yolov8n.pt")  # Load YOLOv8 model
data_yaml_path = r"C:\Users\mdfra\Downloads\deepfake-detection\deepfake-training\data.yaml"  # Path to data.yaml file

# Train YOLOv8 model with custom dataset
model.train(data=data_yaml_path, epochs=5, imgsz=320, batch=8)


KeyboardInterrupt: 